#### CELL-1 코드리뷰 

- <strong>NAME = ''</strong> -> 가져올 모델의 이름을 입력합니다 *전 과정에서 지정했던 이름으로 입력합니다


In [1]:
# Set Hyper Parameters
NAME = '' #모델의 이름을 입력합니다 예시: '12'

<br></br>

#### CELL-2 코드리뷰 

- 필요한 라이브러리들을 가져옵니다, 코드리뷰는 주석으로 대신합니다


In [ ]:
import jajucha2  # 'jajucha2' 모듈을 가져옵니다

import torch  # PyTorch 라이브러리를 가져옵니다. PyTorch는 딥러닝 모델을 구축하고 학습시키는 데 사용되는 라이브러리입니다.

import numpy as np  # NumPy 라이브러리를 가져옵니다. NumPy는 다차원 배열과 수학적 연산을 지원하는 라이브러리입니다.

import torchvision.transforms as transforms  # torchvision.transforms 모듈을 가져옵니다. 이미지 변환에 필요한 여러 가지 도구들을 제공합니다.

from PIL import Image as Img  # PIL (Python Imaging Library)에서 Image 클래스를 가져오고 이를 'Img'라는 이름으로 사용합니다. 이미지 파일을 열고, 조작하고, 저장하는 데 사용됩니다.

import cv2  # OpenCV 라이브러리를 가져옵니다. OpenCV는 컴퓨터 비전 작업을 위한 라이브러리로, 이미지 및 비디오 처리에 널리 사용됩니다.

<br></br>

#### CELL-3 코드리뷰 

- 인공지능 모델을 가져옵니다, 코드리뷰는 주석으로 대신합니다


In [ ]:
device = torch.device('cuda')  # 'cuda'를 사용하여 GPU를 사용할 수 있는지 확인하고, 가능하다면 GPU 장치를 지정합니다. 그렇지 않으면 기본적으로 CPU를 사용하게 됩니다.

model = jajucha2.ai.get_model(NAME)  # 'jajucha2' 모듈의 'ai' 서브모듈에서 'get_model' 함수를 호출하여 인공지능 모델을 가져옵니다.

model.to(device)  # 모델을 지정된 장치(device)로 이동시킵니다. GPU로 모델을 옮겨 계산을 가속화 합니다

model.eval()  # 모델을 평가(evaluation) 모드로 전환합니다. 이는 학습 모드(훈련 모드)에서 사용되는 드롭아웃(dropout)이나 배치 정규화(batch normalization)와 같은 기능들이 평가 시에는 비활성화되도록 하기 위함입니다.

<br></br>

#### CELL-4 코드리뷰 

- <strong>steer = 0</strong> -> 조향값을 초기화 합니다
- <strong>speed = 0</strong> -> 속도값을 초기화 합니다
- 규칙기반, 거리감지 , 신호등을 종합하여 조향값과 속도값을 최종적으로 결정할 것이기 때문에 미리 변수를 생성합니다


In [ ]:
steer = 0
speed = 0

<br></br>

#### CELL-5 코드리뷰 

- 이미지를 가져오고 그리드 기반 알고리즘을 통해 조향각을 도출합니다


In [ ]:
image = jajucha2.camera.get_image('center')

#그리드 기반 주행 코드
(V,L,R) ,grid = jajucha2.camera.gridFront(image)

if(V[5] < 100):
    steer = -10
    speed = 5
else:
    steer = 0
    speed = 5

<br></br>

#### CELL-6 코드리뷰 

- 깊이 이미지를 통해 장애물과의 거리를 계산 후 속도를 결정합니다

In [ ]:
#깊이기반 거리감지 주행
depth = jajucha2.camera.get_depth() 
jajucha2.camera.show_image(depth, 'depth')  
height, width = depth.shape[:2]
center_x, center_y = width // 2, height // 2
region_size = 60
start_x = center_x - region_size // 2  
start_y = center_y - region_size // 2  
center_region = depth[start_y:start_y + region_size, start_x:start_x + region_size]
mean_value = np.mean(center_region)
if(mean_value > 100):
    speed = 0

<br></br>

#### CELL-7 코드리뷰 

- 인공지능 모델에 이미지를 넣어 값을 출력합니다 , 출력된 값에따라 속도를 설정합니다

In [ ]:
## 인공지능 기반 신호등 감지
image = Img.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) 
image = transform(image)  
image =image.unsqueeze(0).to(device)
output = (model(image))
max_index = torch.argmax(output)
if(max_index == 1):
    speed = 0

<br></br>

#### CELL-8 코드리뷰 

- <strong>print(steer, speed)</strong> -> 최종적으로 값을 출력합니다

In [ ]:
print(steer, speed)